Data Flow Between Agents

The system's data flow is coordinated by the Task Planner Agent:

1. Initial Flow: Document → Task Planner → Pre-processor
2. Information Extraction: Pre-processor → Context Bank & Task Planner
3. Knowledge Gathering: Task Planner → Knowledge Agent → Context Bank & Task Planner
4. Compliance Analysis: Task Planner → Compliance Checker (accessing Context Bank)
   - If knowledge is insufficient → Knowledge Agent (with the missing fields)
   - If knowledge is sufficient → Check compliance for each clause
5. Conditional Processing:
   - If contradictions: Compliance Checker → Clause Rewriter → Compliance Checker
   - If compliant: Compliance Checker → Task Planner
6. Summarizing Changes: Task Planner → Post-processor
7. Task Completion: Post-processor → Final Output → User


In [ ]:
from langchain_ollama import ChatOllama
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm

In [ ]:
model = ChatOllama(model="llama3.1:latest")

In [ ]:
planner_agent_tools = [
    create_handoff_tool(agent_name="Pre Processor Agent", description="Transfer when pre-processing is needed, it helps to format and clean the input data."),
    create_handoff_tool(agent_name="Knowledge Agent", description="Transfer when knowledge is needed, it helps to retrieve knowledge from the web using websearcher."),
    create_handoff_tool(agent_name="Compliance Checker Agent", description="Transfer when compliance checking is needed, it helps to check legal documents for compliance with regulations."),
    create_handoff_tool(agent_name="Post Processor Agent", description="Transfer when post processing is needed, it helps to format and finalize the output."),
]

planner_agent_node = create_react_agent(
    model,
    planner_agent_tools,
    prompt="""
        You are a Task Planner Agent responsible for coordinating a multi-agent system to analyze legal documents for discrepancies and compliance. Your job is to plan and delegate tasks to specialized agents, track task completion, and dynamically adapt the plan based on the current system state.

        You are aware of the capabilities of the agents and can query or instruct them based on the task at hand. After every task execution, you should validate whether the task was completed successfully. If a task fails or the output is insufficient, you should modify the workflow, reassign the task, or create additional subtasks.

        YOu have access to the following agents/tools:
        - Knowledge Agent: Transfer when knowledge is needed, it helps to retrieve knowledge from the web using websearcher.

        YOU ARE SUPPOSED TO PLAN WHAT AGENTS AND TOOLS TO CALL AND IN WHAT ORDER.
    """,
    name="Planner Agent"
)

In [ ]:
    # create_handoff_tool(agent_name="Knowledge Agent", description="Transfer when knowledge is needed, it helps to retrieve knowledge from the web using websearcher."),

# TODO : Add the other tools for Pre-Processor Agent - Document parser, Text Classifier, Clause Extractor, NER, Context Bank setter
pre_processor_agent_tools = [
    create_handoff_tool(agent_name="Planner Agent", description="Transfer when pre-processing is completed, it helps to plan the next steps in the workflow and delegate tasks."),
]

pre_processor_agent_node = create_react_agent(
    model,
    pre_processor_agent_tools,
    prompt="""
        You are a Pre-Processor Agent responsible for preparing legal documents for analysis. Your job is to preprocess the documents by rewriting clauses, extracting relevant information, and ensuring that the documents are in a suitable format for further analysis.

        You have access to the following agents/tools:
        - Knowledge Agent: Transfer when knowledge is needed, it helps to retrieve knowledge from the web using websearcher.

    """,
    name="Pre Processor Agent"
)

In [ ]:
# TODO : Add the other tools for Knowledge Agent - Web searcher, Context Bank setter and getter

knowledge_agent_tools = [
    create_handoff_tool(agent_name="Planner Agent", description="Transfer to Planner Agent when knowledge has been retrieved and pass a summary back to it.")
]

knowledge_agent_node = create_react_agent(
    model,
    knowledge_agent_tools,
    prompt="""
    """,
    name="Knowledge Agent",
)

In [ ]:
# TODO : Add the other tools for Compliance Checker Agent - Clause Compliance Checker (which contains the Statutory Validator, Precedent Analyzer, Contractual Consistency Engine, Hypergraph Analyzer, Confidence Scorer), Context Bank getter

compliance_checker_agent_tools = [
    create_handoff_tool(agent_name="Knowledge Agent", description="Transfer to Knowledge Agent if more knowledge is needed, it helps to retrieve knowledge from the web using websearcher."),
    create_handoff_tool(agent_name="Clause Rewriter Agent", description="Transfer to Clause Rewriter Agent if a clause has some discrepancy and rewriting is needed, it helps to rewrite non-compliant clauses in the document."),
    create_handoff_tool(agent_name="Planner Agent", description="Transfer to Planner Agent when compliance checking is completed and all clauses are found to be compliant, it helps to plan the next steps in the workflow and delegate tasks.")
]

compliance_checker_agent_node = create_react_agent(
    model,
    compliance_checker_agent_tools,
    prompt="""
    """,
    name="Compliance Checker Agent",
)

In [ ]:
# TODO : Add the other tools for Clause Rewriter Agent -

clause_rewriter_agent_tools = [
    create_handoff_tool(agent_name="Compliance Checker Agent", description="Transfer to Compliance Checker Agent after a non-compliant clause has been rewritten, it helps to check the compliance of the rewritten clause."),
]

clause_rewriter_agent_node = create_react_agent(
    model,
    clause_rewriter_agent_tools,
    prompt="""
    """,
    name="Clause Rewriter Agent",
)

In [ ]:
# TODO : Add the other tools for Post-Processor Agent - Process Summarizer, Context Bank getter

post_processor_agent_tools = [
    create_handoff_tool(agent_name="Planner Agent", description="Transfer to Planner Agent when post-processing is completed, it helps to plan the next steps in the workflow and delegate tasks."),
]

post_processor_agent_node = create_react_agent(
    model,
    post_processor_agent_tools,
    prompt="""
    """,
    name="Post Processor Agent",
)

In [ ]:
checkpointer = InMemorySaver()
workflow = create_swarm(
    [planner_agent_node, pre_processor_agent_node, knowledge_agent_node, compliance_checker_agent_node, clause_rewriter_agent_node, post_processor_agent_node],
    default_active_agent="Planner Agent"
)
app = workflow.compile(checkpointer=checkpointer)

In [ ]:
config = {"configurable": {"thread_id": "1"}}
turn_1 = app.invoke(
    {"messages": [{"role": "user", "content": "Ask the Knowledge Agent to retrieve information and add the number of letters in the response."}]},
    config,
)
print(turn_1)


{'messages': [HumanMessage(content='Ask the Knowledge Agent to retrieve information and add the number of letters in the response.', additional_kwargs={}, response_metadata={}, id='febec566-dff0-4e8e-9f0a-8bca01ff6952'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2025-04-23T05:05:37.358858Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8654509708, 'load_duration': 790174417, 'prompt_eval_count': 500, 'prompt_eval_duration': 6587681375, 'eval_count': 36, 'eval_duration': 1263667500, 'model_name': 'llama3.1:latest'}, name='Planner Agent', id='run-f950143f-be59-49e6-a682-d68be5cef68f-0', tool_calls=[{'name': 'transfer_to_knowledge_agent', 'args': {'additional_parameter': 'add_letter_count=true', 'question': 'What is the answer to this prompt?'}, 'id': '26518899-02c4-4d84-bb8e-380e6d498e76', 'type': 'tool_call'}], usage_metadata={'input_tokens': 500, 'output_tokens': 36, 'total_tokens': 536}), ToolMessage(content='Successfully transferred to Knowledge Agent', name='transfer_to_knowledge_agent', id='04ee8121-52af-471e-80bf-d54fd0e0aff5', tool_call_id='26518899-02c4-4d84-bb8e-380e6d498e76'), AIMessage(content='The Knowledge Agent has responded with the following information:\n\nThe response was: The response was not provided in the output.\n\n The length of the response is 30 characters.', additional_kwargs={}, response_metadata={'model': 'llama3.1:latest', 'created_at': '2025-04-23T05:05:38.996285Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1614470083, 'load_duration': 20393041, 'prompt_eval_count': 124, 'prompt_eval_duration': 500893208, 'eval_count': 34, 'eval_duration': 1089539833, 'model_name': 'llama3.1:latest'}, name='Knowledge Agent', id='run-3d41073f-dfa6-4b8e-becf-5d446f699ced-0', usage_metadata={'input_tokens': 124, 'output_tokens': 34, 'total_tokens': 158})], 'active_agent': 'Knowledge Agent'}


## Integrating the Compliance Checker Tool

Now we'll create a tool that the Compliance Checker Agent can use to analyze legal documents for compliance issues. This tool will leverage the streamlined `check_legal_compliance` function from compliance_checker.py.


In [ ]:
from agents.compliance_checker import check_legal_compliance
from context_bank import ContextBank
from agents.knowledge import RAGKnowledgeAgent
from langgraph.prebuilt import ToolNode

# Initialize the Context Bank if not already initialized
context_bank = ContextBank()

# Initialize the Knowledge Agent for legal information retrieval
knowledge_agent = RAGKnowledgeAgent(
    use_ollama=True,
    model_name="llama3.1:latest",
    context_bank=context_bank
)

# Create a tool that the ComplianceCheckerAgent can use
def compliance_check_tool(clauses, document_metadata):
    """
    Tool for checking legal document clauses for compliance issues.
    
    Args:
        clauses: List of clause dictionaries, each containing 'id' and 'text' keys
        document_metadata: Document metadata (jurisdiction, document_type, etc.)
        
    Returns:
        List of non-compliant clauses with detailed analysis
    """
    return check_legal_compliance(
        clauses=clauses,
        document_metadata=document_metadata,
        context_bank=context_bank,
        knowledge_agent=knowledge_agent,
        use_ollama=True,
        model_name="llama3.1:latest",
        min_confidence=0.75
    )

# Create a tool node for the compliance checker
compliance_tool_node = ToolNode(tools=[compliance_check_tool])

# Update the compliance_checker_agent_tools to include our new tool
compliance_checker_agent_tools = [
    create_handoff_tool(agent_name="Knowledge Agent", description="Transfer to Knowledge Agent if more knowledge is needed, it helps to retrieve knowledge from the web using websearcher."),
    create_handoff_tool(agent_name="Clause Rewriter Agent", description="Transfer to Clause Rewriter Agent if a clause has some discrepancy and rewriting is needed, it helps to rewrite non-compliant clauses in the document."),
    create_handoff_tool(agent_name="Planner Agent", description="Transfer to Planner Agent when compliance checking is completed and all clauses are found to be compliant, it helps to plan the next steps in the workflow and delegate tasks."),
    compliance_tool_node
]

## Example Usage of the Compliance Checker Tool

Here's how to use the compliance checker tool in the workflow:


In [ ]:
# Example of using the compliance checker tool
def example_compliance_check():
    # Sample document metadata
    document_metadata = {
        "document_id": "doc-123",
        "document_type": "contract",
        "jurisdiction": "US"
    }
    
    # Sample clauses to check
    sample_clauses = [
        {
            "id": "clause-1",
            "text": "The party shall deliver the goods within 30 days."
        },
        {
            "id": "clause-2",
            "text": "The goods must be delivered within 45 days of signing."
        },
        {
            "id": "clause-3",
            "text": "Payment will be made within 15 days of delivery."
        }
    ]
    
    # Call the compliance checker tool
    non_compliant_clauses = compliance_check_tool(
        clauses=sample_clauses,
        document_metadata=document_metadata
    )
    
    return non_compliant_clauses

# Uncomment to run the example
# non_compliant_clauses = example_compliance_check()
# non_compliant_clauses

## Setting up the Compliance Checker Agent with the New Tool

Now we update the Compliance Checker Agent to use our new tool through LangGraph:


In [ ]:
# Update the Compliance Checker Agent node with our new tools
compliance_checker_agent_node = create_react_agent(
    model,
    compliance_checker_agent_tools,
    prompt="""
    You are a Compliance Checker Agent responsible for analyzing legal documents for compliance issues.
    
    Your job is to identify legal issues in document clauses, including:
    1. Statutory violations - where clauses conflict with laws or regulations
    2. Precedent issues - where clauses contradict established legal precedents
    3. Internal consistency issues - where clauses contradict each other
    
    For each non-compliant clause, you will provide detailed analysis including:
    - The type of issue (statutory, precedent, or consistency)
    - Severity of the issue (HIGH, MEDIUM, LOW)
    - References to relevant statutes or other clauses
    - Reasoning for why the clause is non-compliant
    - Legal implications of the issue
    
    You have access to the following agents/tools:
    - Knowledge Agent: Transfer when more knowledge is needed about laws, regulations, or precedents
    - Clause Rewriter Agent: Transfer when non-compliant clauses need to be rewritten
    - compliance_check_tool: Use this to analyze clauses for compliance issues
    
    When you receive clauses for analysis, use the compliance_check_tool to perform a comprehensive compliance check.
    """,
    name="Compliance Checker Agent"
)